In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")

In [4]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion,dia,mes,semana,diadelasemana,hora
0,1112257047,NM5M,2018-01-15 16:22:34,15,1,3,Lunes,16
1,1111920714,NM5M,2018-02-06 09:04:50,6,2,6,Martes,9
2,1112346945,NM5M,2018-02-22 09:04:47,22,2,8,Jueves,9
3,1112345547,NM5M,2018-02-22 09:04:59,22,2,8,Jueves,9
4,1112237522,5awk,2018-01-25 18:55:03,25,1,4,Jueves,18


In [5]:
prediccion.head()

,id,idaviso,idpostulante
0,0,739260,6M9ZQR
1,1,739260,6v1xdL
2,2,739260,ezRKm9
3,3,758580,1Q35ej
4,4,758580,EAN4J6


In [6]:
postulaciones=postulaciones[['idaviso','idpostulante']]
postulaciones['sepostulo']=1
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [7]:
leakeage=pd.merge(prediccion, postulaciones, how="inner",on=['idaviso','idpostulante'])
print(len(leakeage))
leakeage.head()

34500


,id,idaviso,idpostulante,sepostulo
0,594,17903700,Yj5q3xX,1
1,609,1000139175,LNKrDLr,1
2,610,1000139175,exv4B8,1
3,637,1000468151,jkMko3r,1
4,640,1000502125,GNoLbRN,1


In [8]:
leakeage=leakeage[['id','sepostulo']]
leakeage.head()

,id,sepostulo
0,594,1
1,609,1
2,610,1
3,637,1
4,640,1


In [9]:
knn_pred=pd.read_csv("../../Data/fiuba_entrenamiento/gian/predicciones/best_knn.csv")
knn_pred.head()

,id,sepostulo
0,0,0.166667
1,1,0.133333
2,2,0.173333
3,3,0.193333
4,4,0.120000


In [10]:
concat=pd.concat([knn_pred, leakeage],axis=0,ignore_index=True)
concat=concat.drop_duplicates('id',keep='last')
concat.sample(100)

,id,sepostulo
45929,1996,0.013333
99402,99354,0.066667
56726,55371,0.120000
71448,41408,0.260000
94695,94017,0.366667
84876,33664,0.226667
130255,84378,1.000000
121593,70251,1.000000
110369,45386,1.000000
87811,60010,0.073333


In [11]:
print(len(concat))

100000


In [12]:
concat.sort_values('id',inplace=True)
concat.head(20)

,id,sepostulo
0,0,0.166667
1,1,0.133333
2,2,0.173333
3,3,0.193333
4,4,0.120000
5,5,0.080000
6,6,0.446667
7,7,0.120000
8,8,0.440000
9,9,0.260000


In [13]:
concat.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/knn_l.csv",index=False)

In [14]:
len(concat[concat['sepostulo'] == 1])

36107